In [1]:
import os
vipshome = r'C:\Users\Kevin\Downloads\vips-dev-8.14\bin'
os.environ['PATH'] = vipshome + ';' + os.environ['PATH']
import pyvips

In [19]:
import numpy as np
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import cv2
import os
from matplotlib import pyplot as plt
from skimage.morphology import remove_small_objects, remove_small_holes
from skimage.color import rgb2gray
from skimage.transform import warp
from skimage.registration import optical_flow_tvl1, optical_flow_ilk
import xml.etree.ElementTree as ET
import pandas as pd
import skimage.measure
import scipy.stats as stats
import time
import shutil
from tqdm import tqdm
from skimage.measure import label, regionprops_table
from natsort import natsorted

In [14]:
### Find tissue composition of all the label/classes defined by the mask that was created by using the MATLAB deeplabv3 segmentation model:

# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3] # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4] # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
desired_names = [x for x in desired_names if x[-3:] == "_he"]
desired_names = [x+".png" for x in desired_names]

In [17]:
# save xlsx for individual masks:
include_white = True # we want white space included in this case
masksrc = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined'
desired_mask_src = [os.path.join(masksrc,x) for x in desired_names]
desired_mask_names = [os.path.basename(x) for x in desired_mask_src]
save_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined\composition_including_white'
class_list = list(['corneum','spinosum','hairshaft','hairfollicle','smoothmuscle','oil','sweat','nerve','bloodvessel','ecm','fat','white'])
if not include_white:
    count_list_ra = np.zeros(shape=11)
count_list_ra = np.zeros(shape=12)
for idx in tqdm(range(0,len(desired_mask_src)),total = len(desired_mask_src),desc="Image processed",colour='red'):
    comps_df = pd.DataFrame(index = class_list)
    print("processing image: {}".format(desired_mask_src[idx]))
    fn, _ = os.path.splitext(desired_mask_names[idx])
    # check if xlsx exists in destination
    if not include_white:
        save_src_xlsx = os.path.join(save_src, fn + ".xlsx")
    save_src_xlsx = os.path.join(save_src, fn + ".xlsx")
    if os.path.exists(save_src_xlsx):
        print("xlsx for image {} already exists, skipping to next".format(desired_mask_src[idx]))
        continue
    try:
        DLarr = Image.open(desired_mask_src[idx])
    except:
        print("Image {} not available, since corrupted.".format(desired_mask_src[idx]))
        continue
    else:
        print("Image {} available, analyzing tissue composition:".format(desired_mask_src[idx]))
    DLarr = np.array(DLarr)
    if not include_white:
        DLarr = DLarr[DLarr < 12] # background 12 shouldn't count in the composition
    DLarr_flatten = DLarr.flatten()
    hist = np.bincount(DLarr_flatten) #dl mask
    hist = hist/np.sum(hist)*100
    count_list = list(hist)
    print(count_list)
    del(count_list[0])
    count_list_ra = count_list_ra + np.array(count_list)
    count_list = [np.round(x,3) for x in count_list]
    ratio_class_dict = dict(zip(class_list,count_list))
    tmp_df = pd.DataFrame.from_dict(ratio_class_dict,orient='index')
    comps_df = pd.concat([comps_df,pd.DataFrame.from_dict(tmp_df)],axis=1)
    comps_df.dropna()
    comps_df.rename(columns = {0:'composition'}, inplace=True)
    if not include_white:
        comps_df = comps_df[:-1]
    comps_df.to_excel(save_src_xlsx)
print(count_list_ra)
print(np.argmin(count_list_ra))

Image processed:   0%|          | 0/25 [00:00<?, ?it/s]

processing image: \\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined\OTS_14684_1_he.png
Image \\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined\OTS_14684_1_he.png available, analyzing tissue composition:


Image processed:   4%|▍         | 1/25 [00:01<00:38,  1.62s/it]

[0.0, 0.5566447839554887, 0.5125837828440344, 0.033352017312364896, 0.031451211474119056, 0.12958060017794942, 0.041987812639752035, 0.04972032296782768, 0.22298900316018738, 0.36801853494849884, 13.50711421979057, 2.9352636786596484, 81.61129403206957]
processing image: \\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined\OTS_14684_2_he.png
Image \\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined\OTS_14684_2_he.png available, analyzing tissue composition:


Image processed:   8%|▊         | 2/25 [00:02<00:31,  1.38s/it]

...

[0.0, 0.9469857804143069, 0.44603988757454016, 0.09869043866621637, 0.08897933072833632, 0.09454987505909368, 0.06392576819183757, 0.08071696966531973, 0.12853312299630518, 0.28396264457977743, 16.983678380175483, 0.4979643267325348, 80.28597347521624]
processing image: \\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined\OTS_14832_15_he.png
Image \\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined\OTS_14832_15_he.png available, analyzing tissue composition:


Image processed:  96%|█████████▌| 24/25 [00:21<00:01,  1.05s/it]

[0.0, 0.1991639993901792, 1.4975637851944141, 0.3677093954258337, 0.1984957293811466, 0.25930025568022036, 0.6138488116504366, 0.20453831529257138, 0.3278510829180463, 0.9751070743235539, 23.628150847123884, 3.5377898243358907, 68.19048087928383]
processing image: \\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined\OTS_14832_16_he.png
Image \\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined\OTS_14832_16_he.png available, analyzing tissue composition:


Image processed: 100%|██████████| 25/25 [00:22<00:00,  1.12it/s]

[0.0, 0.26905538401741214, 0.38333433913764353, 0.08647615188041476, 0.17198668480516527, 0.13324488185669633, 0.2516227431378201, 0.2148320757034025, 0.15093853246612246, 0.24427584594423887, 17.101569349676584, 2.5828750369217572, 78.40978897445274]
[   7.10179157   14.40234075    2.36856403    2.38441471    7.29313678
    3.18021852    3.34875437    7.73712273    8.59893973  342.60508134
  218.15112321 1382.82851227]
2


### As we see above, for non-white space, most common class is ECM by a big margin, and then fat, and then spinosum, and then blood vessel. Hair shaft is the rarest, 155x fewer area than ecm. But with white space, white space is 4x more common that ECM.

### First, we want to trim the number of ECM that we load into training set. To do that, we should calculate and save as pickle each tile and their respective tissue compositions as a pkl file. Code to do this is below:

In [1]:
# Load all the masks in, adjust the coordinates of the mask to real resolution (mask resolution is 1 um), and then match with coordinate of the tile (in their name). Do both for HE and unstained (note the coordinate diff b/w HE and unstained however). For each image opened, calculate tissue composition and save tissue composition as excel file with the name of the image as well.

In [ ]:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3] # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4] # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
desired_names = [x for x in desired_names if x[-3:] == "_he"]
desired_names = [x+".png" for x in desired_names]
masksrc = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE\1um\classification_v9_combined'
desired_mask_src = [os.path.join(masksrc,x) for x in desired_names]
desired_mask_src = natsorted(desired_mask_src)
desired_mask_names = [os.path.splitext(os.path.basename(x))[0] for x in desired_mask_src]
rsf = 1/0.2204
pkl_save_path = r'\\shelter\Kyu\unstain2stain\unstain2stain_tile\train\tile_composition'
class_list = list(['corneum','spinosum','hairshaft','hairfollicle','smoothmuscle','oil','sweat','nerve','bloodvessel','ecm','fat','white'])
desired_train_src = r'\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\HE'
desired_train_path = [os.path.join(desired_train_src,x) for x in os.listdir(desired_train_src)]
desired_train_path = natsorted(desired_train_path)

for idx in tqdm(range(len(desired_mask_src)),total=len(desired_mask_src),desc = 'Masks/Folders processed',colour='red'): # len and idx of mask and he img folder is the same
    masksrc = desired_mask_src[idx]
    mask = Image.open(masksrc)
    save_src_pkl = os.path.join(pkl_save_path,desired_mask_names[idx])
    if not os.path.exists(save_src_pkl):
        os.makedirs(save_src_pkl)
    heimgsrc = [x for x in os.listdir(desired_train_path[idx])]
    heimgsrc = [os.path.join(desired_train_path[idx],x) for x in heimgsrc] # list of all wsi folders in registrated HE
    print("Processing mask {} and image {}".format(masksrc,heimgsrc))
    heimg_folder = heimgsrc[idx]
    heimg_list = [x for x in os.listdir(heimg_folder) if x.endswith(".png")]
    for idx1 in tqdm(range(len(heimg_list)),total = len(heimgsrc), colour='red',desc='Images processed'):
        imgfn, _ = os.path.splitext(os.path.basename(heimgsrc[idx1]))
        tmp = os.path.splitext('1126_1126xy0001.png')[0]
        x = tmp.split("_")[0]
        tmp1 = tmp.split("_")[1]
        y = tmp1.split("xy")[0]
        # alter x,y to match 1um and then use that x,y to read the x,y rectangle size of 1024 1024 of the mask, get compositoin, and save it:
        x = int(round(x/rsf))
        y = int(round(y/rsf))
        img_size = int(round(1024/rsf))
        part_mask = mask[x:x+img_size,y:y+img_size]
        save_src_pkl1 = os.path.join(save_src_pkl, imgfn + ".pkl")
        DLarr_flatten = part_mask.flatten()
        hist = np.bincount(DLarr_flatten) #dl mask
        hist = hist/np.sum(hist)*100
        count_list = list(hist)
        count_list = [np.round(x,3) for x in count_list]
        ratio_class_dict = dict(zip(class_list,count_list))
        tmp_df = pd.DataFrame.from_dict(ratio_class_dict,orient='index')
        tmp_df.rename(columns = {0:'composition'}, inplace=True)
        comps_df.to_pickle(save_src_pkl1)

In [ ]:
"\\shelter\Kyu\unstain2stain\tiles\registrated_tiles\HE\OTS_14684_1_he\1126_4198xy0004.png"

In [ ]:
    try:
        DLarr = Image.open(desired_mask_src[idx])
    except:
        print("Image {} not available, since corrupted.".format(desired_mask_src[idx]))
        continue
    else:
        print("Image {} available, analyzing tissue composition:".format(desired_mask_src[idx]))